In [1]:
from os import path, makedirs
from itertools import product
import numpy as np
import pandas as pd

from brainpipe.system import study
from scipy.stats import ttest_ind
from Ttests_perm_arthur import ttest_perm
from mne.stats import fdr_correction, bonferroni_correction

In [2]:
###################################################
x0,y0,z0 = 20, 30,-16 #VOI for olf OFC
x1,y1,z1 = -20, 32,-16 #VOI for olf OFC
x0_ins,y0_ins,z0_ins = 42.0, -7.3, 0.3 #VOI for olf Ins
x1_ins,y1_ins,z1_ins = -38, -4, 12 #VOI for olf Ins
rad = 11 #radius of the VOI
HC_lim = -26 #limit anterior/posterior part of the aHC

def rename_elecs(names,x,y,z):
    
    labels = ['pPirT' if lab in ['Amg-PirT'] else lab for lab in names]

    labels = ['aHC' if lab == 'HC' and y[i] > -26 else lab \
                                  for i,lab in enumerate(labels)]
    
    id_R_OFC = [1 if all([x0-rad<=round(x[i])<=x0+rad, y0-rad<=round(y[i])<=y0+rad,
                    z0-rad<=round(z[i])<=z0+rad]) else 0 for i in range(len(labels))]
    id_L_OFC = [1 if all([x1-rad<=round(x[i])<=x1+rad, y1-rad<=round(y[i])<=y1+rad,
                    z1-rad<=round(z[i])<=z1+rad]) else 0 for i in range(len(labels))]
    idx_OFC = id_R_OFC + id_L_OFC
    labels = ['OFC_olf' if i == 1 else lab for i,lab in zip(idx_OFC,labels)]
    
    id_R_ins = [1 if all([x0_ins-rad<=round(x[i])<=x0_ins+rad, y0_ins-rad<=round(y[i])<=y0_ins+rad,
                    z0_ins-rad<=round(z[i])<=z0_ins+rad]) else 0 for i in range(len(labels))]
    id_L_ins = [1 if all([x1_ins-rad<=round(x[i])<=x1_ins+rad, y1_ins-rad<=round(y[i])<=y1_ins+rad,
                    z1_ins-rad<=round(z[i])<=z1_ins+rad]) else 0 for i in range(len(labels))]
    idx_ins = id_R_ins + id_L_ins
    labels = ['Ins_olf' if i == 1 else lab for i,lab in zip(idx_ins,labels)]

    return np.array(labels)

In [73]:
st = study('Olfacto')
exp = 'Enc_Ret'#'Enc'
fold = 'E_R'
perf, meth = 'btw', 'BTW'
conds, subjects = ['high','low'],['VACJ','SEMC','LEFC','PIRJ','FERJ','CHAF']
freqs = ['theta']
path_pow = path.join(st.path, 'feature/TPSim_'+exp+'_By_Odor_By_Cond/TPS_by_cond/6freqs/')
filename = path.join(path_pow, 'TPS_spear_{}_cond_{}_{}_6freqs_3s_dissim.npz')
#path_pow = path.join(st.path, 'feature/TPSim_'+exp+'_By_Odor_By_Cond/TPS_by_odor/')
#filename = path.join(path_pow, 'TPS_pears_{}_{}_{}.npz')
#filename = path.join(path_pow, 'TPS_pears_{}_{}_{}_{}_'+meth+'_med_split.npz')
save_path = path.join(st.path, 'feature/TPSim_'+exp+'_By_Odor_By_Cond/TPS_by_odor/')
df_name = path.join(save_path, '{}_Ttests_old_{}_{}_{}_{}.csv') #su, conds0, conds1, freq
#df_name = path.join(path_pow, '{}_Ttests_'+meth+'_'+perf+'_{}_{}_{}.csv') #su, conds0, conds1, freq
#df_name = path.join(path_pow, '{}_Ttests_'+perf+'_{}_{}_{}.csv') #su, conds0, conds1, freq
nperm = 1000

rois_sel = ['aHC','MFG','ACC','IFG','Amg','pPirT','PHG','Ins_olf','OFC_olf','SFG']

for freq in freqs:
    subjects_c, elecs_c, labels_c = np.array([]), np.array([]), np.array([])
    channels_c, x_c, y_c, z_c = np.array([]), np.array([]), np.array([]), np.array([])
    tps0_c, tps1_c = np.array([]), np.array([])
    T_vals_c, p_vals_c, p_max_c = np.array([]), np.array([]), np.array([])
    p_fdr_c, p_bf_c = np.array([]), np.array([])
    
    for su in subjects:
        if path.exists(filename.format(su,conds[0],freq)):
            mat0 = np.load(filename.format(su,conds[0],freq),allow_pickle=True)
            #mat0b = np.load(filename.format(su,conds[1],perf[0],freq))['tps']
            #tps0 = np.concatenate((mat0['tps'],mat0b),axis=-1)
            tps0 = mat0['tps']
            labels, channels = mat0['label'], mat0['channel']
            x, y, z = mat0['xyz'][:,0], mat0['xyz'][:,1], mat0['xyz'][:,2]
            tps1 = np.load(filename.format(su,conds[1],freq))['tps']
            #mat1b = np.load(filename.format(su,conds[1],perf[1],freq))['tps']
            #tps1 = np.concatenate((mat1,mat1b),axis=-1)
            print(np.mean(tps0),np.mean(tps1))
            print (su,mat0.files, 'TPS shape: ', tps0.shape, tps1.shape)
            
            #rename electrodes labels and select only specific electrodes
            labels_new = rename_elecs(labels,x,y,z)
            idx_sel = [i for i,lab in enumerate(labels_new) if lab in rois_sel]
            tps0, labels, channels = tps0[idx_sel,:], labels_new[idx_sel], channels[idx_sel]
            x, y, z, tps1 = x[idx_sel], y[idx_sel], z[idx_sel], tps1[idx_sel,:]
            nelecs = len(idx_sel)
            
            #compute stats Ttests-unpaired
            tps0, tps1 = tps0.swapaxes(0,1), tps1.swapaxes(0,1) #ntrials x nelecs
            Tvals, unc_p = ttest_ind(tps0, tps1, equal_var=False)
            Tvals2, pvals = ttest_perm(tps0, tps1, n_perm=nperm, correction='maxstat',
                                  two_tailed=False, paired=False, equal_var=False, n_jobs=-1)
            _, p_fdr = fdr_correction(unc_p)
            _, p_bf = bonferroni_correction(unc_p)
            #print(Tvals,unc_p, p_fdr)
            #print(Tvals2,pvals)

            #Fill the csv file with elec infos and stats
            subjects_c = np.hstack((subjects_c,np.array([su]*nelecs))) if np.size(subjects_c) else np.array([su]*nelecs)
            elecs_c = np.hstack((elecs_c,np.arange(nelecs))) if np.size(elecs_c) else np.arange(nelecs)
            labels_c = np.hstack((labels_c,labels)) if np.size(labels_c) else labels
            channels_c = np.hstack((channels_c,channels)) if np.size(channels_c) else channels
            x_c = np.hstack((x_c,x)) if np.size(x_c) else x
            y_c = np.hstack((y_c,y)) if np.size(y_c) else y
            z_c = np.hstack((z_c,z)) if np.size(z_c) else z
            tps0_c = np.hstack((tps0_c,np.mean(tps0, axis=0))) if np.size(tps0_c) else np.mean(tps0, axis=0)
            tps1_c = np.hstack((tps1_c,np.mean(tps1, axis=0))) if np.size(tps1_c) else np.mean(tps1, axis=0)
            T_vals_c = np.hstack((T_vals_c,Tvals)) if np.size(T_vals_c) else Tvals
            p_vals_c = np.hstack((p_vals_c,unc_p)) if np.size(p_vals_c) else unc_p
            p_max_c = np.hstack((p_max_c,pvals)) if np.size(p_max_c) else pvals
            p_fdr_c = np.hstack((p_fdr_c,p_fdr)) if np.size(p_fdr_c) else p_fdr
            p_bf_c = np.hstack((p_bf_c,p_bf)) if np.size(p_bf_c) else p_bf
        
    data = np.concatenate((subjects_c[:,np.newaxis],labels_c[:,np.newaxis],channels_c[:,np.newaxis],x_c[:,np.newaxis],y_c[:,np.newaxis],z_c[:,np.newaxis],elecs_c[:,np.newaxis],
                          tps0_c[:,np.newaxis], tps1_c[:,np.newaxis], T_vals_c[:,np.newaxis],
                           p_vals_c[:,np.newaxis], p_max_c[:,np.newaxis], p_fdr_c[:,np.newaxis],
                          p_bf_c[:,np.newaxis]), axis=1)
    df = pd.DataFrame(data, columns=['subjects','labels','channels','x','y','z',
                            'elecs_num', 'tps_'+conds[0], 'tps_'+conds[1], 'Tvals', 'unc_p', 'max_p', 
                            'fdr_p', 'bonf_p'])
    print(df.shape)
    df.to_csv(df_name.format('All_subjects',conds[0],perf,conds[1],freq),index=False)           


-> Olfacto loaded
0.025278227210930215 0.020477859799328094
VACJ ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  (48, 132) (48, 132)
0.027658502810104946 0.01785139923073272
SEMC ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  (62, 81) (62, 465)
0.00726556844465632 0.004365265448658761
LEFC ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  (36, 252) (36, 408)
0.8983979239555274 0.9036151860255727
PIRJ ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  (30, 176) (30, 168)
0.9146123164882075 0.9351694224007469
FERJ ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  (45, 144) (45, 228)
0.02214815248478383 -0.006229468740445851
CHAF ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  (57, 180) (57, 72)
(232, 14)


In [83]:
from collections import Counter

fold = 'Enc_Ret'
perf, meth = 'btw', 'BTW'
conds = ['high','low']
olf_regions = ['Amg','pPirT','OFC_olf','Ins_olf']

save_path = path.join(st.path, 'feature/TPSim_'+fold+'_By_Odor_By_Cond/TPS_by_odor/')
#save_path = path.join(st.path, 'feature/TPSim_'+fold+'_By_Odor_By_Cond/TPS_by_cond/6freqs/')
#save_path = path.join(st.path, 'classified/TPSim_classif_'+fold+'_by_cond_dissim/')
df_name = path.join(save_path, '{}_Ttests_old_{}_'+perf+'_{}_{}.csv') #su, conds0, conds1, freq
df_stat_save = path.join(save_path, 'stats_df/','{}_Ttests_old_'+meth+'_{}_{}_{}_{}_{}.csv')
#df_name = path.join(save_path, '{}_Ttests_{}_{}_{}.csv') #su, conds0, conds1, freq

df = pd.read_csv(df_name.format('All_subjects',conds[0], conds[1],'theta'))
print('Initial df shape', df.shape)

thrs = [0.05, 0.01]
corrections = ['fdr_p','bonf_p']

for th, corr in product(thrs,corrections):
    df_sel = df.loc[df[corr]<th]
    df_sel['sign'] = ['separation' if t > 0 else 'completion' for t in df_sel['Tvals']]
    print('\n stats at p < ',th, 'correction : ',corr, df_sel.shape)
    print(Counter(df_sel['labels']))

    if fold == 'Enc_Ret' and meth == 'BTW':
        df_sel['tps_high'] = 1-df_sel['tps_high']
        df_sel['tps_low'] =  1-df_sel['tps_low']
        
    rois = np.unique(df_sel['labels'])
    for roi in rois:
        df_roi = df_sel.loc[df_sel['labels']==roi]
        df_inc = df_roi.loc[df_roi['sign']=='completion'].groupby(['subjects']).count()
        df_dec = df_roi.loc[df_roi['sign']=='separation'].groupby(['subjects']).count()
            
        if (df_inc.shape[0] >= 3) or (df_inc.shape[0] >=2 and roi in olf_regions):
            print(roi, 'NB of subjects with completion',df_inc.shape[0],' subjects')
            df_plot = df_roi.loc[df_roi['sign']=='completion']
            print('#electrodes in total >>> ',df_plot.shape[0])
            df_plot.to_csv(df_stat_save.format('All_subjects',conds[0],
                                conds[1],'theta',roi,corr+str(th)))
            
        if (df_dec.shape[0] >= 3) or (df_dec.shape[0] >=2 and roi in olf_regions):
            print(roi, 'NB of subjects with separation',df_dec.shape[0],' subjects')
            df_plot = df_roi.loc[df_roi['sign']=='separation']
            print('#electrodes in total >>> ',df_plot.shape[0])
            df_plot.to_csv(df_stat_save.format('All_subjects',conds[0],
                                conds[1],'theta',roi,corr+str(th)))
            #print(df_plot)


Initial df shape (232, 14)

 stats at p <  0.05 correction :  fdr_p (51, 15)
Counter({'SFG': 11, 'MFG': 10, 'aHC': 10, 'IFG': 6, 'OFC_olf': 4, 'Amg': 3, 'PHG': 3, 'pPirT': 2, 'ACC': 2})
IFG NB of subjects with separation 3  subjects
#electrodes in total >>>  4
MFG NB of subjects with separation 3  subjects
#electrodes in total >>>  6
OFC_olf NB of subjects with completion 3  subjects
#electrodes in total >>>  3
aHC NB of subjects with completion 3  subjects
#electrodes in total >>>  8
pPirT NB of subjects with completion 2  subjects
#electrodes in total >>>  2

 stats at p <  0.05 correction :  bonf_p (32, 15)
Counter({'aHC': 7, 'MFG': 5, 'IFG': 4, 'SFG': 4, 'Amg': 3, 'PHG': 3, 'OFC_olf': 3, 'pPirT': 2, 'ACC': 1})
MFG NB of subjects with separation 3  subjects
#electrodes in total >>>  5
OFC_olf NB of subjects with completion 2  subjects
#electrodes in total >>>  2
pPirT NB of subjects with completion 2  subjects
#electrodes in total >>>  2

 stats at p <  0.01 correction :  fdr_p (32,

In [21]:
path = '/media/karim/Datas4To/1_Analyses_Intra_EM_Odor/Olfacto/feature/TPSim_Enc_By_Odor_By_Cond/TPS_by_odor/'

mat = np.load(path+'TPS_pears_SEMC_high_wth_theta.npz')
chan = [i for i,elec in enumerate(mat['channel']) if elec == 'b5-b4']
tpsim = mat['tps'][chan,...]
print(tpsim.shape)
print(np.mean(tpsim))

(1, 9)
1.022779643814247
